<a href="https://colab.research.google.com/github/WilliamCori/I-Escuela-Latam-Observatorio-Virtual-SVO-2023/blob/main/VO_with_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center> VO with Python: working with PyVO and MOCPy </center></h1>

Created by

**Pedro Mas Buitrago, [SVO](https://svo.cab.inta-csic.es/main/index.php)**

**Last updated** by the author in March 2023

This tutorial has taken some references from the adaptation of "[Advanced usage of HiPS and MOCs to explore complex regions of interest](https://github.com/cds-astro/tutorials/tree/master/Notebooks)" to jupyter notebook by the Strasbourg astronomical Data Center ([CDS](https://cdsweb.u-strasbg.fr/)) team.

For the tutorial to work correctly, the user must upload [these data files](https://cloud.cab.inta-csic.es/index.php/s/YggXCpJp6wHkJ5B) in their own Google Colab session. To do this, click on the "Files" icon on the left side of the page, select the "Upload to session storage" option (first icon in the pop-up menu) and upload the files from the local directory.

---

<h2><center> Technical background </center></h2>

The Multi-Order Coverage map (**MOC**) is a Virtual Observatory method created to specify any kind of sky region in a simple and efficient way. As mentioned in the [reference document](https://www.ivoa.net/documents/MOC/20220727/REC-moc-2.0-20220727.pdf), the rationale behind MOC is to get a method to manipulate sky coverages in order to provide very fast union, intersection and equality operations between them.

The MOC method is based on the [HEALPix](https://iopscience.iop.org/article/10.1086/427976) tesselation technique, a method of partitioning the sphere that consists of diamond cells with equal spherical surfaces. The HEALPix tessellation technique divides the sphere into 12 cells, each of them sub-divided into 4 cells recursively (see Fig. 1). Thus the sphere at order 1 will consist of 48 cells, 192 cells at order 2, 768 at order 3 and so on where each cell at a given order is covering an equal area of the sphere.

In [ ]:
from IPython.display import Image
Image('healpix.png',width=950, height=250)

<center> Fig. 1: HEALPix partition of the sphere. The corresponding HEALPix order is 0, 1, 2 and 3, from left to right.  </center>

---

<h2><center> Before we get started </center></h2>

This is a hands-on tutorial demonstrating the usage of PyVO and MOCPy, two products for accessing the VO from Python, in which we will:

1. Learn to access astronomical data using PyVO
2. Work with Multi-Order Coverage (MOC) maps from different surveys using MOCPy
3. See how the use of MOCs can benefit a real scientific case
4. Retrieve objects from large catalogs using a specific MOC

We will use the python packages included in the file `requirements.txt`. The python packages include [MOCPy](https://github.com/cds-astro/mocpy), [astroquery](https://astroquery.readthedocs.io/en/latest/), [PyVO](https://pyvo.readthedocs.io/en/latest/), [astropy](https://docs.astropy.org/en/stable/index.html), [pandas](https://pandas.pydata.org/docs/), [numpy](https://numpy.org/doc/stable/) and [matplotlib](https://matplotlib.org/stable/index.html).

To install these packages:

In [ ]:
! pip install -r /content/requirements.txt

First of all, we will import these packages!

In [ ]:
import warnings
import pandas as pd
import numpy as np
import pyvo
import matplotlib.pyplot as plt
from mocpy import MOC, World2ScreenMPL

from astropy.coordinates import Angle, SkyCoord
import astropy.units as u
from astroquery.vizier import Vizier
import plotly.express as px

We will also make use of the [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR) service at [CDS](https://cdsweb.u-strasbg.fr/), which allows you to look for astronomical catalogues that have been published in the literature. Some of these catalogs contain data, associated with the publications, that can be consulted through the VizieR-associated data service.

---

<h2><center> Accesing data with PyVO </center></h2>

`PyVO` is an affiliated package for the `astropy` package that lets you find and retrieve astronomical data available from archives that support standard IVOA virtual observatory service protocols. In this section, we will explore the Table Access Protocol ([TAP](https://www.ivoa.net/documents/TAP/20190927/REC-TAP-1.1.pdf)), which allows access to source catalogs using [ADQL](http://docs.g-vo.org/adql/html/index.html) queries.

All TAP services must support a set of tables in a schema named TAP_SCHEMA that describes the tables and columns included in the service. In particular, the table TAP_SCHEMA.tables must contain the following columns:

<center>

| Column name | Type | not-null |
| :-: | :-: | :-: |
| schema_name | string | true |
| table_name | string | true |
| table_type | string | true |
| utype | string | false |
| description | string | false |
| table_index | integer | false |

</center>

In our case, we will use PyVO to retrieve data from the Gaia data release 3. To search for Gaia data, we will use the TAP with the Gaia endpoint. First, let's see which schemas (and information) are available in the gaia database.

In [ ]:
tap_gaia = pyvo.dal.TAPService("https://gea.esac.esa.int/tap-server/tap")

query = '''
SELECT DISTINCT schema_name, table_type
FROM tap_schema.tables
'''

gaia_info = tap_gaia.search(query).to_table()

gaia_info

> In a database, a **View** is a virtual table whose content is defined by a query to one or more tables. The main advantage of a view is that it can join data from several tables, avoiding repeating complex queries.

For this tutorial, we are interested in the tables from the schema `gaiadr3` containing the information about the observed sources, so we search for them:

In [ ]:
tap_gaia = pyvo.dal.TAPService("https://gea.esac.esa.int/tap-server/tap")

query = '''
SELECT *
FROM tap_schema.tables
WHERE schema_name = 'gaiadr3'
AND table_name LIKE '%source%'
AND table_type = 'table'
'''

gaia_info = tap_gaia.search(query).to_table()
gaia_info

As you can see in the description, the table we are looking for is `gaiadr3.gaia_source`. As a practical example, let's search for Pleiades-comoving objects with reliable parallax and proper motion (relative error less than 10$\%$).

In [ ]:
query = '''
SELECT TOP 40000 *
FROM gaiadr3.gaia_source
WHERE ABS(pmra_error/pmra) < 0.1
AND ABS(pmdec_error/pmdec) < 0.1
AND pm < 100
AND parallax_over_error > 10
AND 1=CONTAINS(
  POINT(ra,dec),
  CIRCLE(56.601,24.114,5))
'''

gaia_data = tap_gaia.search(query)

gaia_data.to_table()

With the attribute `fieldnames`, we can check the fields included in the output table:

In [ ]:
gaia_data.fieldnames

Rich metadata equivalent to what is found in VOTables (including unit, ucd and datatype) is available through resultset’s `getdesc()` method.

> In a VOTable, the attribute **ucd** stands for Unified Content Descriptor, and supplies a standardized classification of the physical quantity expressed in the column. 

In [ ]:
gaia_data.getdesc('parallax')

Now that we can build the proper motion diagram to locate the comoving sources.

For a proper manipulation of the data, it is useful to create a pandas dataframe with the information required. In this cell, we will see how we can create this dataframe from the information contained in the output table.

In [ ]:
col_names = ['source_id','ra','dec','pmra','pmdec','pm']
data_list = []

for col in col_names:
    
    obj = gaia_data[col]
    
    x_ = obj.data.reshape(len(obj),1)
    
    data_list.append(x_)
        
gaia_data_df = pd.DataFrame(np.concatenate(data_list,axis=1),columns=col_names)
gaia_data_df['source_id'] = gaia_data_df['source_id'].astype(int)

gaia_data_df

In [ ]:
fig = px.scatter(gaia_data,x='pmra',y='pmdec',template='none',color_discrete_sequence=['darkred'],width=800, height=600)
fig.update_traces(marker=dict(size=5,
                              color='rgba(164, 0, 0,0.35)'))

fig.show()

---

<center><h2> Working with MOCs in Python </h2></center>

MOCPy is a Python library that allows the manipulation of Multi-Order Coverage maps (MOCs) in a straightforward way. It is possible to load the MOCs from different sources, such as FITS files, urls or VizieR tables. Some powerful tasks that we can perform thanks to MOCPY is to compare the sky coverage of different surveys or to compute the union/intersection between them.

In this section we will explore both. First, we will see how the sky coverage of the Sloan Digital Sky Survey, [SDSS](https://www.sdss.org/), has changed over the years. Then, we will compute the union and intersection between the sky coverage of the SDSS data release 16 and the second data release of the [J-PLUS](http://www.j-plus.es/datareleases/data_release_dr2) survey.

First, we need to search for the VizieR tables that store the data corresponding to the data releases 3, 6 and 16 of SDSS. One way to search for the SDSS catalogues available on VizieR is to use the `Vizier` module in the `astroquery` package.

In [ ]:
catalog_list = Vizier.find_catalogs('SDSS DR3')
for k, v in catalog_list.items():
    print(k, ': ', v.description)

The catalog we are looking for is `II/259 :  The SDSS Photometric Catalog, Release 3 (Abazajian+ 2005)`. To see the tables included in this catalog, we can use the `get_catalogs()` method:

In [ ]:
table_list = Vizier.get_catalogs('II/259')

table_list

***Exercise 1:*** *Find the corresponding tables for the remaining SDSS data releases, SDSS DR6 and SDSS DR16.*



In [ ]:
#@title
catalog_list = Vizier.find_catalogs('SDSS DR6')
for k, v in catalog_list.items():
    print(k, ': ', v.description)

warnings.filterwarnings('ignore')

In [ ]:
#@title
table_list = Vizier.get_catalogs('II/282')

table_list

In [ ]:
#@title
catalog_list = Vizier.find_catalogs('SDSS DR16')
for k, v in catalog_list.items():
    print(k, ': ', v.description)

warnings.filterwarnings('ignore')

In [ ]:
#@title
table_list = Vizier.get_catalogs('V/154')

table_list

The tables we need are `II/259/sdss3`, `II/282/sdss6` and `V/154/sdss16`. Now, we load the MOCs for the different SDSS data releases using these tables.

In [ ]:
moc_sdssdr3 = MOC.from_vizier_table('II/259/sdss3')
moc_sdssdr6 = MOC.from_vizier_table('II/282/sdss6')
moc_sdssdr16 = MOC.from_vizier_table('V/154/sdss16')

To plot the MOCs, we first need to use the `World2ScreenMPL` MOCPy class to create the astropy Word Coordinate System (WCS) that represents the field of view of the figure, as explained in the [MOCPy](https://cds-astro.github.io/mocpy/examples/examples.html) documentation.

> World Coordinate Systems (WCSs) describe the geometric transformations between one set of coordinates and another. A common application is to map the pixels in an image onto the celestial sphere.

In [ ]:
fig = plt.figure(111,figsize=(15,9))

with World2ScreenMPL(fig,
                    fov = 340*u.deg,
                    center=SkyCoord(0,0,unit='deg',frame='icrs'),
                    coordsys='icrs', rotation=Angle(0, u.degree),
                    projection='AIT') as wcs:
    
    ax = fig.add_subplot(1,1,1,projection=wcs)

    moc_sdssdr16.fill(ax=ax, wcs=wcs, alpha=0.15, fill=True,color='#003f5c', label= 'SDSS DR16 (2020)')
    moc_sdssdr16.border(ax=ax, wcs=wcs, alpha=0.4, color='#003f5c')   
    
    moc_sdssdr6.fill(ax=ax, wcs=wcs, alpha=0.4, fill=True, color='#ffa600', label='SDSS DR6 (2007)')
    moc_sdssdr6.border(ax=ax, wcs=wcs, alpha=0.6, color='#ffa600')
    
    moc_sdssdr3.border(ax=ax, wcs=wcs, color='#a05195', lw=2, label='SDSS DR3 (2005)')

ax.legend(fontsize=12)
plt.grid(color='black', linestyle='dotted')

plt.show()

Now, we will plot the intersection of SDSS DR16 and J-PLUS DR2. For the latter, we will load the MOC from the corresponding FITS file, and then compute the intersection between both using the `intersection` MOCPy method.

In [ ]:
moc_jplusdr2 = MOC.from_fits("/content/jplus_dr2_moc.fits")

***Exercise 2:*** *Plot the J-PLUS DR2 MOC.*

In [ ]:
#@title
fig = plt.figure(111,figsize=(15,9))

with World2ScreenMPL(fig,
                    fov = 340*u.deg,
                    center=SkyCoord(0,0,unit='deg',frame='icrs'),
                    coordsys='icrs',rotation=Angle(0, u.degree),
                    projection='AIT') as wcs:
    
    ax = fig.add_subplot(1,1,1,projection=wcs)
    
    moc_jplusdr2.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True,color='#ffa600')
    moc_jplusdr2.border(ax=ax, wcs=wcs, alpha=0.8,color='#ffa600')

ax.set_xlabel('RA',fontsize=16)
ax.set_ylabel('DEC',fontsize=16)
plt.title('J-PLUS DR2 sky coverage',fontsize=14)
plt.grid(color='black',linestyle='dotted')

plt.show()

In [ ]:
inter = moc_sdssdr16.intersection(moc_jplusdr2)

In [ ]:
fig = plt.figure(111,figsize=(15,9))

with World2ScreenMPL(fig,
                    fov = 260*u.deg,
                    center=SkyCoord(0,0,unit='deg',frame='icrs'),
                    coordsys='icrs',rotation=Angle(0, u.degree),
                    projection='AIT') as wcs:
    
    ax = fig.add_subplot(1,1,1,projection=wcs)
    
    inter.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True,color='#003f5c')
    inter.border(ax=ax, wcs=wcs, alpha=0.8,color='#003f5c')      

ax.set_xlabel('RA',fontsize=16)
ax.set_ylabel('DEC',fontsize=16)
plt.title('Intersection between SDSS DR16 and J-PLUS DR2',fontsize=14)
plt.grid(color='black',linestyle='dotted')

plt.show()

***Exercise 3:*** *Compute and plot the union between SDSS DR16 and J-PLUS DR2*

In [ ]:
#@title
union = moc_sdssdr16.union(moc_jplusdr2)

fig = plt.figure(111,figsize=(15,9))

with World2ScreenMPL(fig,
                    fov = 260*u.deg,
                    center=SkyCoord(0,0,unit='deg',frame='icrs'),
                    coordsys='icrs',rotation=Angle(0, u.degree),
                    projection='AIT') as wcs:
    
    ax = fig.add_subplot(1,1,1,projection=wcs)
    
    union.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True,color='#ffa600')
    union.border(ax=ax, wcs=wcs, alpha=0.8,color='#ffa600')      

ax.set_xlabel('RA',fontsize=16)
ax.set_ylabel('DEC',fontsize=16)
plt.title('Union between SDSS DR16 and J-PLUS DR2',fontsize=14)
plt.grid(color='black',linestyle='dotted')

plt.show()

warnings.filterwarnings("ignore", category=DeprecationWarning)

---

<h2><center> Real scientific case with MOCs </center></h2>

In this section, we will explore a real example in with the use of MOCs contributed to refine the scientific results of a [paper](https://arxiv.org/abs/2208.09377). For this, we will use the data from the [SVO archive of ultracool dwarfs identified in J-PLUS DR2](http://svocats.cab.inta-csic.es/jplus_ucds1/index.php?action=search), which contains information for the 9.810 candidate ultracool dwarfs identified using Virtual Observatory tools in the entire sky coverage of J-PLUS DR2.

In astronomy, when working with photometry from different surveys, you have to be sure that the photometry coming from each of them for the same filters/wavelengths is equivalent. This becomes very important when, for example, you are using the photometry of the stars to infer physical parameters such as the effective temperature. In our particular case, the use of the SDSS and J-PLUS DR2 MOCs allowed us to see that there was a significant difference between the density of objects identified in the regions where both MOCs overlapped and in regions where they did not. Looking further into this, we found a systematic difference between the photometry in SDSS and J-PLUS DR2 for the same filters that had to be corrected.

To reproduce this case, we will only need the coordinates of the objects, that are included in the file `UCD_data.csv`.

In [ ]:
data = pd.read_csv('UCD_data.csv')

data

When working with celestial coordinates, it is good practice to define them using the `SkyCoord` class of the `astropy` package, which provides a flexible interface for celestial coordinate representation, manipulation, and transformation between systems.

In [ ]:
coords = SkyCoord(data[['RA','DEC']].values,frame='icrs',unit='deg')
coords

We now plot the corresponding MOCs, including our objects in top of them.

In [ ]:
fig = plt.figure(111, figsize=(15,9))

with World2ScreenMPL(fig, 
        fov=260* u.deg,
        center=SkyCoord(0, 0, unit='deg', frame='icrs'),
        coordsys="icrs",
        rotation=Angle(0, u.degree),
        projection="AIT") as wcs:
    
    ax = fig.add_subplot(1, 1, 1, projection=wcs)
    
    moc_sdssdr16.fill(ax=ax, wcs=wcs, alpha=0.15, fill=True, color='#003f5c',label='SDSS DR16')
    moc_sdssdr16.border(ax=ax, wcs=wcs, alpha=0.4, color='#003f5c')
    
    moc_jplusdr2.fill(ax=ax, wcs=wcs, alpha=0.4, fill=True, color="#ffa600",label='J-PLUS DR2')
    moc_jplusdr2.border(ax=ax, wcs=wcs, alpha=0.6, color="#ffa600")
    
    ax.scatter(coords.ra,coords.dec,marker='o',color='#003f5c',s=0.3,transform=ax.get_transform('world'))

ax.legend(fontsize=15)
ax.set_xlabel('RA',fontsize=16)
ax.set_ylabel('DEC',fontsize=16)
plt.grid(color="black", ls='dotted')

plt.show()

Zooming into a region where both MOCs overlap, we can check whether the density of objects is the same as in the non-overlapping regions.

In [ ]:
fig = plt.figure(111, figsize=(15,11))

with World2ScreenMPL(fig, 
        fov=30 * u.deg,
        center=SkyCoord(273,41, unit='deg', frame='icrs'),
        coordsys="icrs",
        rotation=Angle(0, u.degree),
        projection="AIT") as wcs:
    
    ax = fig.add_subplot(1, 1, 1, projection=wcs)
    
    moc_sdssdr16.fill(ax=ax, wcs=wcs, alpha=0.05, fill=True, color='#003f5c',label='SDSS DR16')
    moc_sdssdr16.border(ax=ax, wcs=wcs, alpha=0.5, color='#003f5c')
    
    moc_jplusdr2.fill(ax=ax, wcs=wcs, alpha=0.2, fill=True, color='#ffa600',label='J-PLUS DR2')
    moc_jplusdr2.border(ax=ax, wcs=wcs, alpha=1, color='#ffa600')
    
    ax.scatter(coords.ra,coords.dec,marker='o',color='#003f5c',s=3,transform=ax.get_transform('world'))

ax.set_xlabel('RA',fontsize=14)
ax.set_ylabel('DEC',fontsize=14)
ax.legend(fontsize=18)
plt.grid(color="black", linestyle="dotted")

plt.show()

---

<h2><center> Query the 2MASS Catalogue by MOC </center></h2>
    
Without the usage of MOC, querying for sources in a specific region of the sky would be very tedious. Here, we will use the power of MOC files to query large catalogs directly in the covered regions only. We will use the MOC of the [MiniJ-PAS Public Data Release](http://www.j-pas.org/datareleases/minijpas_public_data_release_pdr201912), which covers a total area of $~\,1\,deg^2$, to query for sources from the 2MASS survey.

In [ ]:
moc_jpas = MOC.from_fits('/content/mini_jpas_moc.fits')

In [ ]:
fig = plt.figure(111,figsize=(15,9))

with World2ScreenMPL(fig,
                    fov = 10*u.deg,
                    center=SkyCoord(214,52.5,unit='deg',frame='icrs'),
                    coordsys='icrs',rotation=Angle(0, u.degree),
                    projection='AIT') as wcs:
    
    ax = fig.add_subplot(1,1,1,projection=wcs)
    
    moc_jpas.fill(ax=ax, wcs=wcs, alpha=0.5, fill='true',color='#ffa600')
    moc_jpas.border(ax=ax, wcs=wcs, alpha=0.8,color='#ffa600')
    
ax.set_xlabel('RA',fontsize=16)
ax.set_ylabel('DEC',fontsize=16)
plt.title('Sky coverage of MiniJ-PAS Public Data Release',fontsize=18)
plt.grid(color='black',linestyle='dotted')

plt.show()

To search for the table from the 2MASS survey containing the desired information, we will use the TAP protocol with the VizieR endpoint.

Through the VizieR TAP endpoint, we can explore the VizieR associated data service to search for tables, content of tables or information on data associated to the existing catalogues.

In [ ]:
tap_vizier = pyvo.dal.TAPService('http://tapvizier.u-strasbg.fr/TAPVizieR/tap')

query = '''
SELECT  *  FROM tap_schema.tables 
WHERE description LIKE '2MASS %source%'
'''

twomass_tables = tap_vizier.search(query).to_table()
twomass_tables

The table we are looking for is `II/246/out`, which stores the 2MASS Point Source Catalog. We can query this table to have a look at its content.

In [ ]:
query = '''
SELECT Top 5 * FROM "II/246/out" 
'''

twomass_source_table = tap_vizier.search(query).to_table()
twomass_source_table

***Exercise 4:*** *Another way to search for the 2MASS catalogues available on VizieR is to use the `Vizier` module in the `astroquery` package, as we did before. Find the desired table `II/246/out` using this module.*

In [ ]:
#@title
catalog_list_twomass = Vizier.find_catalogs('2MASS source')
for k, v in catalog_list_twomass.items():
    print(k, ': ', v.description)

warnings.filterwarnings('ignore')

In [ ]:
#@title
table_list_twomass = Vizier.get_catalogs('II/246')

print(table_list_twomass)

Once we have found the target table `II/246/out`, we can use the `query_vizier_table` MOCPy method to query this table and search for sources in the coverage of the MOC instance.

In [ ]:
twomass_sources = moc_jpas.query_vizier_table('II/246/out')

twomass_sources

As seen below, the search has been performed accurately throughout the entire sky coverage of MiniJ-PAS Public Data Release.

In [ ]:
coords = SkyCoord(twomass_sources['RAJ2000'].data,twomass_sources['DEJ2000'].data,frame="icrs", unit="deg")

fig = plt.figure(111,figsize=(15,9))

with World2ScreenMPL(fig,
                    fov = 6*u.deg,
                    center=SkyCoord(214,52.5,unit='deg',frame='icrs'),
                    coordsys='icrs',rotation=Angle(0, u.degree),
                    projection='AIT') as wcs:
    
    ax = fig.add_subplot(1,1,1,projection=wcs)
    
    moc_jpas.fill(ax=ax, wcs=wcs, alpha=0.2, fill='true',color='#ffa600')
    moc_jpas.border(ax=ax, wcs=wcs, alpha=1,color='#ffa600')
    

ax.scatter(twomass_sources['RAJ2000'],twomass_sources['DEJ2000'],marker='o',s=1,c='#003f5c',transform=ax.get_transform('world'))

ax.set_xlabel('RA',fontsize=16)
ax.set_ylabel('DEC',fontsize=16)
plt.grid(color='black',linestyle='dotted')

plt.show()